In [1]:
import sys
!{sys.executable} -m pip install hyperopt

You are using pip version 9.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [5]:
import hyperopt

In [6]:
import sys
!{sys.executable} -m pip install lightgbm

You are using pip version 9.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
import lightgbm

In [9]:
import sys
!{sys.executable} -m pip install xgboost

You are using pip version 9.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [1]:
import xgboost

PermissionError: [WinError 5] Access is denied

# Creating a function for K-folds cross validation using hyperopt

In [2]:
data_path = "C:/Users/as14478/Sanchita Kaggle/"
submission_path= data_path+'submission/'
fold_path = data_path+'fold_data/'

from Functions import *

from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
import lightgbm as lgb

%matplotlib inline
seed=1204

cv_loss_list=[]
n_iteration_list=[]

def score(params):
    print("Training with params: ")
    print(params)
    cv_losses=[]
    cv_iteration=[]
    for (train_idx,val_idx) in cv:
        cv_train = X.iloc[train_idx]
        cv_val = X.iloc[val_idx]
        cv_y_train = y[train_idx]
        cv_y_val = y[val_idx]
        lgb_model = lgb.train(params, lgb.Dataset(cv_train, label=cv_y_train), 2000, 
                          lgb.Dataset(cv_val, label=cv_y_val), verbose_eval=False, 
                          early_stopping_rounds=100)
       
        train_pred = lgb_model.predict(cv_train,lgb_model.best_iteration+1)
        val_pred = lgb_model.predict(cv_val,lgb_model.best_iteration+1)
        
        val_loss = root_mean_squared_error(cv_y_val,val_pred)
        train_loss = root_mean_squared_error(cv_y_train,train_pred)
        print('Train RMSE: {}. Val RMSE: {}'.format(train_loss,val_loss))
        print('Best iteration: {}'.format(lgb_model.best_iteration))
        cv_losses.append(val_loss)
        cv_iteration.append(lgb_model.best_iteration)
    print('6 fold results: {}'.format(cv_losses))
    cv_loss_list.append(cv_losses)
    n_iteration_list.append(cv_iteration)
    
    mean_cv_loss = np.mean(cv_losses)
    print('Average iterations: {}'.format(np.mean(cv_iteration)))
    print("Mean Cross Validation RMSE: {}\n".format(mean_cv_loss))
    return {'loss': mean_cv_loss, 'status': STATUS_OK}

def optimize(space,seed=seed,max_evals=5):
    
    best = fmin(score, space, algo=tpe.suggest, 
        # trials=trials, 
        max_evals=max_evals)
    return best

C:\Users\sp14657\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


# 6-fold validation (28th to 33rd month)

In [ ]:
all_data = get_all_data(data_path,'new_sales_lag_after12.pickle')


In [19]:
all_data = get_all_data(data_path,'new_sales_lag_after12.pickle')
X,y = get_X_y(all_data,33)
X.drop('date_block_num',axis=1,inplace=True)
cv = get_cv_idxs(all_data,28,33)

In [21]:
space = {
#     'max_depth': hp.choice('max_depth', np.arange(3, 15, dtype=int)),
    'subsample': hp.quniform('subsample', 0.5, 1, 0.05),
    'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.05),
    'min_data_in_leaf': hp.choice('min_data_in_leaf',np.arange(5, 30,1, dtype=int)),
    'learning_rate': hp.quniform('learning_rate', 0.025, 0.5, 0.025),
    'seed':seed,
    'objective': 'regression',
    'metric':'rmse',
}
best_hyperparams = optimize(space,max_evals=3)
print("The best hyperparameters are: ")
print(best_hyperparams)

Training with params: 
{'colsample_bytree': 0.75, 'learning_rate': 0.17500000000000002, 'metric': 'rmse', 'min_data_in_leaf': 22, 'objective': 'regression', 'seed': 1204, 'subsample': 0.75}
Train RMSE: 0.8065677366834108. Val RMSE: 0.8494131223854234
Best iteration: 62
Train RMSE: 0.7565263643872259. Val RMSE: 0.7733251782609203
Best iteration: 210
Train RMSE: 0.736246454701719. Val RMSE: 0.7031075830034939
Best iteration: 320
Train RMSE: 0.7216661784228514. Val RMSE: 0.7721335170388263
Best iteration: 438
Train RMSE: 0.7579030289397541. Val RMSE: 0.8900546099778947
Best iteration: 199
Train RMSE: 0.7694341485475304. Val RMSE: 0.9029543455622276
Best iteration: 159
6 fold results: [0.8494131223854234, 0.7733251782609203, 0.7031075830034939, 0.7721335170388263, 0.8900546099778947, 0.9029543455622276]
Average iterations: 231.33333333333334
Mean Cross Validation RMSE: 0.8151647260381311

Training with params: 
{'colsample_bytree': 0.8, 'learning_rate': 0.225, 'metric': 'rmse', 'min_data_i

# Training and then predicting on test dataset

In [23]:
all_data = get_all_data(data_path,'new_sales_lag_after12.pickle')

X,y = get_X_y(all_data,33)
X.drop('date_block_num',axis=1,inplace=True)

cv = get_cv_idxs(all_data,28,33)

In [24]:
lgb_params = {
               'colsample_bytree': 0.75,
               'metric': 'rmse',
               'min_data_in_leaf': 17, 
               'subsample': 0.75, 
               'learning_rate': 0.17500000000000002, 
               'objective': 'regression', 
               'bagging_seed': 128, 
               'num_leaves': 128,
               'bagging_freq':1,
               'seed':1204
              }

In [25]:
%%time
lgb_model_full = lgb.train(lgb_params, lgb.Dataset(X, label=y), 708, 
                      lgb.Dataset(X, label=y), verbose_eval=10)

[10]	valid_0's rmse: 0.834014
[20]	valid_0's rmse: 0.787916
[30]	valid_0's rmse: 0.770406
[40]	valid_0's rmse: 0.757848
[50]	valid_0's rmse: 0.749162
[60]	valid_0's rmse: 0.741375
[70]	valid_0's rmse: 0.734258
[80]	valid_0's rmse: 0.728912
[90]	valid_0's rmse: 0.724566
[100]	valid_0's rmse: 0.719468
[110]	valid_0's rmse: 0.715095
[120]	valid_0's rmse: 0.712043
[130]	valid_0's rmse: 0.707965
[140]	valid_0's rmse: 0.704797
[150]	valid_0's rmse: 0.700684
[160]	valid_0's rmse: 0.697294
[170]	valid_0's rmse: 0.693418
[180]	valid_0's rmse: 0.690801
[190]	valid_0's rmse: 0.68851
[200]	valid_0's rmse: 0.685305
[210]	valid_0's rmse: 0.683543
[220]	valid_0's rmse: 0.680558
[230]	valid_0's rmse: 0.677783
[240]	valid_0's rmse: 0.67543
[250]	valid_0's rmse: 0.673806
[260]	valid_0's rmse: 0.670363
[270]	valid_0's rmse: 0.668387
[280]	valid_0's rmse: 0.667029
[290]	valid_0's rmse: 0.665002
[300]	valid_0's rmse: 0.663148
[310]	valid_0's rmse: 0.660914
[320]	valid_0's rmse: 0.658788
[330]	valid_0's rms

In [35]:
test = pd.read_csv(os.path.join(data_path, 'test.csv'))
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [36]:
test.shape

(214200, 3)

In [39]:
all_data.head()

,shop_id,item_id,date_block_num,target,item_name,item_category_id,shop_block_target_sum,shop_block_target_mean,item_block_target_sum,item_block_target_mean,...,shop_block_target_mean_lag_12,shop_block_target_sum_lag_12,item_cat_block_target_mean_lag_12,item_cat_block_target_sum_lag_12,target_lag_12,Newyear_Xmas,Valentine_MenDay,WomenDay,Easter_Labor,December
0,54,10297,12,4.0,ГАДКИЙ Я 1-2 (BD),37,8198.0,1.295717,23.0,1.000000,...,1.247143,9386.0,1.053414,6094.0,0.0,1,0,0,0,0
1,54,10296,12,3.0,ГАДКИЙ Я 1-2 (3D BD),38,8198.0,1.295717,17.0,1.000000,...,1.247143,9386.0,1.019387,1630.0,0.0,1,0,0,0,0
2,54,10298,12,14.0,ГАДКИЙ Я 2,40,8198.0,1.295717,182.0,1.181818,...,1.247143,9386.0,1.145903,33489.0,0.0,1,0,0,0,0
3,54,10300,12,3.0,ГАДКИЙ Я 2 (BD),37,8198.0,1.295717,26.0,0.962963,...,1.247143,9386.0,1.053414,6094.0,0.0,1,0,0,0,0
4,54,10284,12,1.0,"Высоцкий Владимир Спасибо, что живой (mp3-CD)...",57,8198.0,1.295717,3.0,1.000000,...,1.247143,9386.0,1.003914,1539.0,0.0,1,0,0,0,0


In [42]:
print(all_data['date_block_num'].min())
print(all_data['date_block_num'].max())

12
33


In [52]:
sum(all_data['date_block_num'].apply(lambda x : x ==33))

238172

# Creating test dataset

In [86]:
all_data = pd.read_csv(data_path+'new_sales_v1_reduced.csv')
test = pd.read_csv('test.csv')
items = pd.read_csv('items.csv')

In [87]:
all_data.head()
list(all_data)

['shop_id',
 'item_id',
 'date_block_num',
 'target',
 'item_name',
 'item_category_id',
 'shop_block_target_sum',
 'shop_block_target_mean',
 'item_block_target_sum',
 'item_block_target_mean',
 'item_cat_block_target_sum',
 'item_cat_block_target_mean']

In [88]:
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [89]:
test =pd.merge(test,items,on=['item_id'],how='left')
test.head()
list(test)

['ID', 'shop_id', 'item_id', 'item_name', 'item_category_id']

In [90]:
test['target'] = 0 
print(test)
test['shop_block_target_sum'] = 0 
print(test)
test['shop_block_target_mean'] = 0 
print(test)
test['item_block_target_sum'] = 0 
print(test)
test['item_block_target_mean'] = 0 
print(test)
test['item_cat_block_target_sum'] = 0 
print(test)
test['item_cat_block_target_mean'] = 0 


            ID  shop_id  item_id  \
0            0        5     5037   
1            1        5     5320   
2            2        5     5233   
3            3        5     5232   
4            4        5     5268   
5            5        5     5039   
6            6        5     5041   
7            7        5     5046   
8            8        5     5319   
9            9        5     5003   
10          10        5     4806   
11          11        5     4843   
12          12        5     4607   
13          13        5     4869   
14          14        5     4870   
15          15        5     4872   
16          16        5     4874   
17          17        5     4678   
18          18        5     4892   
19          19        5     4964   
20          20        5     4717   
21          21        5     5002   
22          22        5     5823   
23          23        5     5814   
24          24        5     5900   
25          25        5     5907   
26          26        5     

In [91]:
test['date_block_num'] = 34
print(test)

            ID  shop_id  item_id  \
0            0        5     5037   
1            1        5     5320   
2            2        5     5233   
3            3        5     5232   
4            4        5     5268   
5            5        5     5039   
6            6        5     5041   
7            7        5     5046   
8            8        5     5319   
9            9        5     5003   
10          10        5     4806   
11          11        5     4843   
12          12        5     4607   
13          13        5     4869   
14          14        5     4870   
15          15        5     4872   
16          16        5     4874   
17          17        5     4678   
18          18        5     4892   
19          19        5     4964   
20          20        5     4717   
21          21        5     5002   
22          22        5     5823   
23          23        5     5814   
24          24        5     5900   
25          25        5     5907   
26          26        5     

In [92]:
print (all_data.shape)
print (test.shape) 
print (list(all_data))
print (list(test))


(10913804, 12)
(214200, 13)
['shop_id', 'item_id', 'date_block_num', 'target', 'item_name', 'item_category_id', 'shop_block_target_sum', 'shop_block_target_mean', 'item_block_target_sum', 'item_block_target_mean', 'item_cat_block_target_sum', 'item_cat_block_target_mean']
['ID', 'shop_id', 'item_id', 'item_name', 'item_category_id', 'target', 'shop_block_target_sum', 'shop_block_target_mean', 'item_block_target_sum', 'item_block_target_mean', 'item_cat_block_target_sum', 'item_cat_block_target_mean', 'date_block_num']


In [96]:
all_data = pd.concat([all_data,test])

In [97]:
all_data.head()

,ID,date_block_num,item_block_target_mean,item_block_target_sum,item_cat_block_target_mean,item_cat_block_target_sum,item_category_id,item_id,item_name,shop_block_target_mean,shop_block_target_sum,shop_id,target
0,NaN,0.0,1.0,18.0,1.053414,6094.0,37,22154,ЯВЛЕНИЕ 2012 (BD),1.092041,2017.0,59,1.0
1,NaN,0.0,0.0,0.0,0.969595,287.0,58,2552,DEEP PURPLE The House Of Blue Light LP,1.092041,2017.0,59,0.0
2,NaN,0.0,1.0,1.0,0.969595,287.0,58,2554,DEEP PURPLE Who Do You Think We Are LP,1.092041,2017.0,59,0.0
3,NaN,0.0,1.0,2.0,1.007519,268.0,56,2555,DEEP PURPLE 30 Very Best Of 2CD (Фирм.),1.092041,2017.0,59,0.0
4,NaN,0.0,1.0,5.0,0.990113,701.0,59,2564,DEEP PURPLE Perihelion: Live In Concert DVD (К...,1.092041,2017.0,59,0.0


In [101]:
all_data.shape
index_cols = ['shop_id', 'item_id', 'date_block_num','item_category_id']
cols_to_rename = list(all_data.columns.difference(index_cols))
for i in ['item_name']:
    cols_to_rename.remove(i)
print(cols_to_rename)
cols_gb_item = [i for i in cols_to_rename if 'item_block' in i]
cols_gb_shop = [i for i in cols_to_rename if 'shop_block' in i]
cols_gb_cat = [i for i in cols_to_rename if 'item_cat' in i]
cols_gb_all = ['target']
cols_gb_key=[['item_id'],['shop_id'],['item_category_id'],['shop_id','item_id']]
cols_gb_value = [cols_gb_item,cols_gb_shop,cols_gb_cat,cols_gb_all]
print(cols_gb_value)
shift_range = [1,2,3,5,12]
for month_shift in shift_range:
    for k,v in zip(cols_gb_key,cols_gb_value): 
        index_col = ['date_block_num'] + k
        train_shift = all_data[index_col + v].copy().drop_duplicates()

        train_shift['date_block_num'] = train_shift['date_block_num'] + month_shift

        foo = lambda x: '{}_lag_{}'.format(x, month_shift) if x in v else x
        train_shift = train_shift.rename(columns=foo)
        all_data = pd.merge(all_data, train_shift, on=index_col, how='left').fillna(0)

['ID', 'item_block_target_mean', 'item_block_target_sum', 'item_cat_block_target_mean', 'item_cat_block_target_sum', 'shop_block_target_mean', 'shop_block_target_sum', 'target']
[['item_block_target_mean', 'item_block_target_sum'], ['shop_block_target_mean', 'shop_block_target_sum'], ['item_cat_block_target_mean', 'item_cat_block_target_sum'], ['target']]


In [102]:
all_data.to_csv(data_path + 'test_all_lag_data.csv',index=False)

In [103]:
all_datadata.head()

,ID,date_block_num,item_block_target_mean,item_block_target_sum,item_cat_block_target_mean,item_cat_block_target_sum,item_category_id,item_id,item_name,shop_block_target_mean,...,item_cat_block_target_mean_lag_5,item_cat_block_target_sum_lag_5,target_lag_5,item_block_target_mean_lag_12,item_block_target_sum_lag_12,shop_block_target_mean_lag_12,shop_block_target_sum_lag_12,item_cat_block_target_mean_lag_12,item_cat_block_target_sum_lag_12,target_lag_12
0,0.0,0.0,1.0,18.0,1.053414,6094.0,37,22154,ЯВЛЕНИЕ 2012 (BD),1.092041,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.969595,287.0,58,2552,DEEP PURPLE The House Of Blue Light LP,1.092041,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,1.0,0.969595,287.0,58,2554,DEEP PURPLE Who Do You Think We Are LP,1.092041,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,2.0,1.007519,268.0,56,2555,DEEP PURPLE 30 Very Best Of 2CD (Фирм.),1.092041,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,5.0,0.990113,701.0,59,2564,DEEP PURPLE Perihelion: Live In Concert DVD (К...,1.092041,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [104]:
test_lag = all_data[all_data.date_block_num==34].copy()
test_lag.to_csv(data_path + 'test_lag_data.csv',index=False)

In [112]:
test_lag.head()

,item_block_target_mean,item_block_target_sum,item_cat_block_target_mean,item_cat_block_target_sum,item_category_id,item_id,shop_block_target_mean,shop_block_target_sum,shop_id,target,...,item_cat_block_target_mean_lag_5,item_cat_block_target_sum_lag_5,target_lag_5,item_block_target_mean_lag_12,item_block_target_sum_lag_12,shop_block_target_mean_lag_12,shop_block_target_sum_lag_12,item_cat_block_target_mean_lag_12,item_cat_block_target_sum_lag_12,target_lag_12
16234852,0.0,0.0,0.0,0.0,19,5037,0.0,0.0,5,0.0,...,1.067340,3487.0,1.0,1.0,65.0,1.235043,1445.0,1.144403,6134.0,1.0
16234853,0.0,0.0,0.0,0.0,55,5320,0.0,0.0,5,0.0,...,1.016900,6017.0,0.0,0.0,0.0,1.235043,1445.0,1.041406,9809.0,0.0
16234854,0.0,0.0,0.0,0.0,19,5233,0.0,0.0,5,0.0,...,1.067340,3487.0,2.0,0.0,0.0,1.235043,1445.0,1.144403,6134.0,0.0
16234855,0.0,0.0,0.0,0.0,23,5232,0.0,0.0,5,0.0,...,1.044452,2984.0,0.0,0.0,0.0,1.235043,1445.0,1.099875,5275.0,0.0
16234856,0.0,0.0,0.0,0.0,20,5268,0.0,0.0,5,0.0,...,1.381781,4687.0,0.0,0.0,0.0,1.235043,1445.0,2.338131,12834.0,0.0


# Predicting on test dataset

In [114]:
test_lag = pd.read_csv(os.path.join(data_path, 'test_lag_data.csv'))
test_lag.drop(['ID','item_name','date_block_num'],axis=1,inplace=True)
test_lag_pred = lgb_model_full.predict(test_lag,708)

In [123]:
def get_submission(item_cnt_month,sub_name,clip=20,data_path ='C:/Users/as14478/Sanchita Kaggle/' ):
    item_cnt_month = np.clip(item_cnt_month,0,clip)
    test= pd.read_csv(os.path.join(data_path, 'test.csv.gz'))
    sub = test.copy()
    sub['item_cnt_month'] = item_cnt_month
    sub.drop(['item_id','shop_id'],axis=1,inplace=True)
    sub.to_csv(data_path+'submission/' + sub_name+'.csv',index=False)
    return sub
get_submission(test_lag_pred,'lightgbm_basic_6folds');

In [125]:
test_lag_temp = pd.read_csv(os.path.join(data_path, 'test_lag_data.csv'), encoding = "ISO-8859-1")

In [126]:
test_lag_temp.head()

,ID,date_block_num,item_block_target_mean,item_block_target_sum,item_cat_block_target_mean,item_cat_block_target_sum,item_category_id,item_id,item_name,shop_block_target_mean,...,item_cat_block_target_mean_lag_5,item_cat_block_target_sum_lag_5,target_lag_5,item_block_target_mean_lag_12,item_block_target_sum_lag_12,shop_block_target_mean_lag_12,shop_block_target_sum_lag_12,item_cat_block_target_mean_lag_12,item_cat_block_target_sum_lag_12,target_lag_12
0,0.0,34.0,0.0,0.0,0.0,0.0,19,5037,"NHL 15 [PS3, ??????? ????????]",0.0,...,1.067340,3487.0,1.0,1.0,65.0,1.235043,1445.0,1.144403,6134.0,1.0
1,1.0,34.0,0.0,0.0,0.0,0.0,55,5320,ONE DIRECTION Made In The A.M.,0.0,...,1.016900,6017.0,0.0,0.0,0.0,1.235043,1445.0,1.041406,9809.0,0.0
2,2.0,34.0,0.0,0.0,0.0,0.0,19,5233,"Need for Speed Rivals (Essentials) [PS3, ?????...",0.0,...,1.067340,3487.0,2.0,0.0,0.0,1.235043,1445.0,1.144403,6134.0,0.0
3,3.0,34.0,0.0,0.0,0.0,0.0,23,5232,"Need for Speed Rivals (Classics) [Xbox 360, ??...",0.0,...,1.044452,2984.0,0.0,0.0,0.0,1.235043,1445.0,1.099875,5275.0,0.0
4,4.0,34.0,0.0,0.0,0.0,0.0,20,5268,"Need for Speed [PS4, ??????? ??????]",0.0,...,1.381781,4687.0,0.0,0.0,0.0,1.235043,1445.0,2.338131,12834.0,0.0


# Adding Holiday Features to test dataset

In [8]:
import pandas as pd
import os
data_path = "C:/Users/sp14657/all/From Drive/"
test_lag_temp = pd.read_csv(os.path.join(data_path, 'test_lag_data.csv'), encoding = "ISO-8859-1")
test_lag_temp.head()

test_lag_temp['Newyear_Xmas'] = test_lag_temp.date_block_num.apply(lambda x: 1 if x in [12,24] else 0)
test_lag_temp['Valentine_MenDay'] = test_lag_temp.date_block_num.apply(lambda x: 1 if x in [13,25] else 0)
test_lag_temp['WomenDay'] = test_lag_temp.date_block_num.apply(lambda x: 1 if x in [14,26] else 0)
test_lag_temp['Easter_Labor'] = test_lag_temp.date_block_num.apply(lambda x: 1 if x in [15,27] else 0)
test_lag_temp['December'] = test_lag_temp.date_block_num.apply(lambda x: 1 if x ==23 else 0)

test_lag_temp.to_csv(data_path + 'test_lag_data_Holiday_features.csv',index=False)

In [9]:
test_lag_temp

,ID,date_block_num,item_block_target_mean,item_block_target_sum,item_cat_block_target_mean,item_cat_block_target_sum,item_category_id,item_id,item_name,shop_block_target_mean,...,shop_block_target_mean_lag_12,shop_block_target_sum_lag_12,item_cat_block_target_mean_lag_12,item_cat_block_target_sum_lag_12,target_lag_12,Newyear_Xmas,Valentine_MenDay,WomenDay,Easter_Labor,December
0,0.0,34.0,0.0,0.0,0.0,0.0,19,5037,"NHL 15 [PS3, ??????? ????????]",0.0,...,1.235043,1445.0,1.144403,6134.0,1.0,0,0,0,0,0
1,1.0,34.0,0.0,0.0,0.0,0.0,55,5320,ONE DIRECTION Made In The A.M.,0.0,...,1.235043,1445.0,1.041406,9809.0,0.0,0,0,0,0,0
2,2.0,34.0,0.0,0.0,0.0,0.0,19,5233,"Need for Speed Rivals (Essentials) [PS3, ?????...",0.0,...,1.235043,1445.0,1.144403,6134.0,0.0,0,0,0,0,0
3,3.0,34.0,0.0,0.0,0.0,0.0,23,5232,"Need for Speed Rivals (Classics) [Xbox 360, ??...",0.0,...,1.235043,1445.0,1.099875,5275.0,0.0,0,0,0,0,0
4,4.0,34.0,0.0,0.0,0.0,0.0,20,5268,"Need for Speed [PS4, ??????? ??????]",0.0,...,1.235043,1445.0,2.338131,12834.0,0.0,0,0,0,0,0
5,5.0,34.0,0.0,0.0,0.0,0.0,23,5039,"NHL 15 [Xbox 360, ??????? ????????]",0.0,...,1.235043,1445.0,1.099875,5275.0,0.0,0,0,0,0,0
6,6.0,34.0,0.0,0.0,0.0,0.0,20,5041,"NHL 16 [PS4, ??????? ????????]",0.0,...,1.235043,1445.0,2.338131,12834.0,0.0,0,0,0,0,0
7,7.0,34.0,0.0,0.0,0.0,0.0,55,5046,NICKELBACK No Fixed Address,0.0,...,1.235043,1445.0,1.041406,9809.0,1.0,0,0,0,0,0
8,8.0,34.0,0.0,0.0,0.0,0.0,55,5319,ONE DIRECTION Four,0.0,...,1.235043,1445.0,1.041406,9809.0,5.0,0,0,0,0,0
9,9.0,34.0,0.0,0.0,0.0,0.0,20,5003,"NBA 2K16 [PS4, ??????? ????????????]",0.0,...,1.235043,1445.0,2.338131,12834.0,0.0,0,0,0,0,0


## Predicting on revised test set (with holiday features)

In [3]:
data_path = "C:/Users/sp14657/all/From Drive/"
from Functions import *
test_lag = pd.read_csv(os.path.join(data_path, 'test_lag_data_Holiday_features.csv'), encoding = "ISO-8859-1")

In [4]:
all_data = get_all_data(data_path,'new_sales_lag_after12.pickle')

X,y = get_X_y(all_data,33)
X.drop('date_block_num',axis=1,inplace=True)

cv = get_cv_idxs(all_data,28,33)

lgb_params = {
               'colsample_bytree': 0.75,
               'metric': 'rmse',
               'min_data_in_leaf': 17, 
               'subsample': 0.75, 
               'learning_rate': 0.17500000000000002, 
               'objective': 'regression', 
               'bagging_seed': 128, 
               'num_leaves': 128,
               'bagging_freq':1,
               'seed':1204
              }


In [5]:
%%time
lgb_model_full = lgb.train(lgb_params, lgb.Dataset(X, label=y), 708, 
                      lgb.Dataset(X, label=y), verbose_eval=100)

[100]	valid_0's rmse: 0.717746
[200]	valid_0's rmse: 0.685734
[300]	valid_0's rmse: 0.66312
[400]	valid_0's rmse: 0.643606
[500]	valid_0's rmse: 0.630708
[600]	valid_0's rmse: 0.618307
[700]	valid_0's rmse: 0.608576
Wall time: 8min 23s


In [6]:
test_lag.drop(['ID','item_name','date_block_num'],axis=1,inplace=True)
test_lag_pred = lgb_model_full.predict(test_lag,708)

In [7]:
def get_submission(item_cnt_month,sub_name,clip=20,data_path ='C:/Users/sp14657/all/From Drive/' ):
    item_cnt_month = np.clip(item_cnt_month,0,clip)
    test= pd.read_csv(os.path.join(data_path, 'test.csv.gz'))
    sub = test.copy()
    sub['item_cnt_month'] = item_cnt_month
    sub.drop(['item_id','shop_id'],axis=1,inplace=True)
    sub.to_csv(data_path+'submission/' + sub_name+'.csv',index=False)
    return sub

get_submission(test_lag_pred,'lightgbm_basic_6folds - revised test v2');

In [8]:
test_lag.head()

,item_block_target_mean,item_block_target_sum,item_cat_block_target_mean,item_cat_block_target_sum,item_category_id,item_id,shop_block_target_mean,shop_block_target_sum,shop_id,target,...,shop_block_target_mean_lag_12,shop_block_target_sum_lag_12,item_cat_block_target_mean_lag_12,item_cat_block_target_sum_lag_12,target_lag_12,Newyear_Xmas,Valentine_MenDay,WomenDay,Easter_Labor,December
0,0.0,0.0,0.0,0.0,19,5037,0.0,0.0,5,0.0,...,1.235043,1445.0,1.144403,6134.0,1.0,0,0,0,0,0
1,0.0,0.0,0.0,0.0,55,5320,0.0,0.0,5,0.0,...,1.235043,1445.0,1.041406,9809.0,0.0,0,0,0,0,0
2,0.0,0.0,0.0,0.0,19,5233,0.0,0.0,5,0.0,...,1.235043,1445.0,1.144403,6134.0,0.0,0,0,0,0,0
3,0.0,0.0,0.0,0.0,23,5232,0.0,0.0,5,0.0,...,1.235043,1445.0,1.099875,5275.0,0.0,0,0,0,0,0
4,0.0,0.0,0.0,0.0,20,5268,0.0,0.0,5,0.0,...,1.235043,1445.0,2.338131,12834.0,0.0,0,0,0,0,0
